# I. Loading/Plot functions

In [6]:
DATASETS = {
    'a': 'dogs',
    'b': 'medical-leaf',
    'c': 'texture-dtd',
    'd': 'birds',
    'e': 'AWA',
    'f': 'plt-net',
    'g': 'resisc',
    'h': 'plt-doc',
    'i': 'airplanes',
    '_': 'ALL'
}

color_dict = {
        'erm': 'blue',
        'jtt': 'red',
        'suby': '#00CC96',  # green
        'subg': '#ff7f0e',  # orange
        'rwy': '#00CC96',
        'rwg': '#ff7f0e',
        'dro': '#DEA0FD'  # purple
    }
    

In [7]:
import wandb
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Initialize wandb
wandb.login()

# Set your entity and project
entity_name = "aureliengauffre"  # e.g., your username or team name
project_name = "SMA_all_2_best"

# Initialize the wandb API ||client
api = wandb.Api()

# Fetch all runs from the specified project
runs = api.runs(f"{entity_name}/{project_name}")

# Create an empty list to hold data for each run
data = []

# Loop through runs and ext/ract the data you're interested in
for run in runs:
    # Extract both summary metrics and config (hyperparameters) for each run
    run_data = {
        "name": run.name,
        "summary_metrics": run.summary._json_dict,
        "config": run.config,
        # Add any other attributes you're interested in here
    }
    data.append(run_data)

# Convert the list of data to a pandas DataFrame
df = pd.DataFrame(data)

# For summary metrics and config (hyperparameters), expand them into separate columns
df_summary = pd.json_normalize(df['summary_metrics'])
df_config = pd.json_normalize(df['config'])
df = pd.concat([df.drop(['summary_metrics', 'config'], axis=1), df_summary, df_config], axis=1)
df = df.iloc[:, 1:]  # Drop the first column containing the name of the runs (since there is also another name column)

# Now we have a DataFrame `df` with all runs, their summary metrics, and hyperparameters
# print(df.head())  # Print the first few rows of the DataFrame


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aureliengauffre (use `wandb login --relogin` to force relogin)


In [8]:
df['name']

0            plt-net
1            plt-net
2                AWA
3            plt-net
4                AWA
            ...     
2963       airplanes
2964       airplanes
2965    medical-leaf
2966       airplanes
2967       airplanes
Name: name, Length: 2968, dtype: object

In [9]:
# Initialize a Plotly figure
def plot_graph_old(df, x_axis, y_axis, dataset_name, methods=None):
    # Default methods if not provided
    
    if methods is None:
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    
    if dataset_name is not None:
        df_dataset = df[(df['name'] == dataset_name) ]#& (df['mu'] >= .1) ]
    else :
        df_dataset = df 
    fig = go.Figure() # Initialize a Plotly figure
    for method in methods:
        df_method = df_dataset[df_dataset['method'] == method]
        # Group by x_axis and calculate the mean of y_axis
        df_avg = df_method.groupby(x_axis)[y_axis].mean().reset_index()
        
        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line to the plot for the current method
        color = color_dict.get(method, 'grey')
        fig.add_trace(go.Scatter(x=df_avg[x_axis], y=df_avg[y_axis], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color)))
    
    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis} , dataset={dataset_name}',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=600   # Height of the figure in pixels
                     )

    # Show the figure
    fig.show()

# Example usage (assuming df is your DataFrame):
# plot_graph(df, 'K', 'mean_gr


dataset_name = 'AWA' #73sports
y_axis = 'mean_grp_acc_te' #'mean_grp_acc_te'
x_axis = 'K'
plot_graph(df, x_axis, y_axis, dataset_name, methods=None)


In [10]:
import plotly.graph_objects as go


def normalize_df(df,metric = 'best_acc_te'):
    # Copy the DataFrame to avoid modifying the original data
    result_df = df.copy()
    
    # Group by 'name', 'K', 'mu', and 'init_seed' and calculate the minimum and maximum accuracy
    grouped = df.groupby(['name', 'K', 'mu', 'init_seed'])[metric]
    min_acc = grouped.transform(np.min)
    max_acc = grouped.transform(np.max)
    
    # Apply the Min-Max normalization formula
    result_df[metric] = (df[metric] - min_acc) / (max_acc - min_acc)
    
    return result_df




def plot_graph(df, x_axis, y_axis, dataset_name, error_bars=None, normalize = False, methods=None,):
    print(dataset_name)
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    color_dict = {
            'erm': 'blue',
            'jtt': 'red',
            'suby': '#00CC96',  # green
            'subg': '#ff7f0e',  # orange
            'rwy': '#00CC96',
            'rwg': '#ff7f0e',
            'dro': '#DEA0FD'  # purple
        }
    if dataset_name is None or dataset_name == 'ALL': 
        df_dataset = df
    else:
        df_dataset = df[(df['name'] == dataset_name)]
    
    if normalize :
        df_dataset = normalize_df(df_dataset,metric=y_axis)
        
    fig = go.Figure()  # Initialize a Plotly figure
    for method in methods:
        df_method = df_dataset[df_dataset['method'] == method]
        # Group by x_axis and calculate the mean, standard deviation, and count (for standard error calculation)
        stats = df_method.groupby(x_axis)[y_axis].agg(['mean', 'std', 'count']).reset_index()
        
        # Calculate standard error (SEM)
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        #print(method, stats) # print the number on which we average, interesting !
        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line with error bars to the plot for the current method
        color = color_dict.get(method, 'grey')
        if error_bars :
            fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['mean'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color),
                                 error_y=dict(type='data', array=stats['sem'], visible=True)))
        else :
            fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['mean'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color)))
    
    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis}, dataset={dataset_name}',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )

    # Show the figure
    
    fig.show()


def plot_graph_all(df, x_axis, y_axis, error_bars=None, normalize=False, methods=None):
    """Unlike the original plot_graph function which calculates and plots error bars
    based on individual datasets directly, this version computes the standard error within each dataset first 
    and then averages these errors across all datasets for each method. This approach provides a generalized 
    view of method performance and variability across different datasets."""
    
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    
    if normalize:
        df = normalize_df(df, metric=y_axis)

    fig = go.Figure()  # Initialize a Plotly figure
    all_stats = pd.DataFrame()

    # Process each method separately
    for method in methods:
        df_method = df[df['method'] == method]

        # Group data first by 'name' and then by x_axis and compute statistics
        grouped = df_method.groupby(['name', x_axis])
        stats = grouped[y_axis].agg(['mean', 'std', 'count']).reset_index()

        # Calculate standard error within each dataset
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        
        # Append results to the all_stats DataFrame for later display
        stats['method'] = method
        all_stats = pd.concat([all_stats, stats], ignore_index=True)
        
        # Now group by x_axis and calculate the mean of the means and the mean of the SEMs
        final_stats = stats.groupby(x_axis).agg({'mean': 'mean', 'sem': 'mean'}).reset_index()

        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line with error bars to the plot for the current method
        color = color_dict.get(method, 'grey')
        if error_bars:
            fig.add_trace(go.Scatter(x=final_stats[x_axis], y=final_stats['mean'], mode='lines+markers',
                                     name=method, line=dict(dash=line_style, color=color),
                                     error_y=dict(type='data', array=final_stats['sem'], visible=True)))
        else:
            fig.add_trace(go.Scatter(x=final_stats[x_axis], y=final_stats['mean'], mode='lines+markers',
                                     name=method, line=dict(dash=line_style, color=color)))

    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis}, Average on all dataset (mean and std)',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )

    # Show the figure
    fig.show()

    # Display the all_stats DataFrame
    return all_stats.pivot_table(index=[x_axis, 'name'], columns='method', values=['count', 'sem'], aggfunc='first')



# II. Analysis : K

Here are the **plot parameters** to be played with :

In [11]:
ERROR_BARS = True # Wether to plot the error bars
NORMALIZE = False

To analyse the impact of K, we the **value of mu is fixed**:

In [12]:
df_fix_mu = df[df['mu']==.2] # Currently mu in [.05,.1,.2,.4]

df_fix_mu_K2 = df[df['K']==2] 
df_fix_mu_K4 = df[df['K']==4] 
df_fix_mu_K8 = df[df['K']==8] 
df_fix_mu_K12 = df[df['K']==12] 

## a. best_acc_te

In [13]:
x_axis = 'K'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)

count                                           sem            \
method           dro   erm   jtt   rwg   rwy  subg  suby       dro       erm   
K name                                                                         
2 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001127  0.002104   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.003954  0.002942   
  birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001567  0.002324   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN  0.004987   
  medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002848  0.002848   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.016838  0.013397   
  plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001480  0.001172   
  resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.005031  0.004786   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.011326  0.015278   
4 AWA            NaN   NaN   5.0   NaN   NaN   NaN   5.0       NaN       NaN   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.002447  0.003824   
  birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000218  0.000588   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN  0.001564   
  medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.001638  0.003804   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.007205  0.005376   
  plt-net        NaN   NaN   5.0   NaN   NaN   NaN   0.0       NaN       NaN   
  resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.010090  0.003310   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.008621  0.007312   
8 AWA            0.0   NaN   5.0   5.0   5.0   NaN   NaN       NaN       NaN   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.000475  0.000865   
  birds          NaN   5.0   5.0   5.0   5.0  10.0  10.0       NaN  0.000182   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN  0.001722   
  medical-leaf  15.0  15.0  15.0  15.0  15.0  15.0  15.0  0.001738  0.002140   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   NaN  0.002201  0.002359   
  plt-net        NaN  10.0   NaN   NaN   NaN   5.0   NaN       NaN  0.000538   
  resisc         3.0   NaN   5.0   5.0   5.0   5.0   5.0  0.005392       NaN   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.006883  0.001137   

                                                                  
method               jtt       rwg       rwy      subg      suby  
K name                                                            
2 AWA           0.002353  0.001743  0.001294  0.000303  0.002906  
  airplanes     0.006266  0.005733  0.004522  0.009099  0.003539  
  birds         0.004976  0.001816  0.003265  0.005415  0.003679  
  dogs          0.005396       NaN       NaN       NaN  0.010999  
  medical-leaf  0.012948  0.002848  0.002326  0.005696  0.006780  
  plt-doc       0.010416  0.013045  0.012522  0.029757  0.015031  
  plt-net       0.002058  0.001649  0.001978  0.001249  0.001473  
  resisc        0.001867  0.003463  0.003421  0.006568  0.003623  
  texture-dtd   0.020880  0.008391  0.014797  0.014467  0.024611  
4 AWA           0.000745       NaN       NaN       NaN  0.000606  
  airplanes     0.003036  0.004953  0.004302  0.003883  0.003374  
  birds         0.000390  0.000369  0.000275  0.000677  0.000906  
  dogs          0.001277       NaN       NaN       NaN  0.002622  
  medical-leaf  0.003374  0.001176  0.002518  0.012856  0.010980  
  plt-doc       0.008013  0.006064  0.003139  0.006292  0.006162  
  plt-net       0.001921       NaN       NaN       NaN       NaN  
  resisc        0.001645  0.002409  0.005807  0.005984  0.004261  
  texture-dtd   0.003446  0.010546  0.011076  0.009626  0.023341  
8 AWA           0.000943  0.000217  0.000735       NaN       NaN  
  airplanes     0.001167  0.001968  0.000638  0.001062  0.000650  
  birds         0.000181  0.000130  0.000363  0.000610  0.000130  
  dogs

In [14]:
x_axis = 'K'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, normalize=NORMALIZE)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## b. worst_acc_te

In [15]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=False, normalize=NORMALIZE)

count                                           sem             \
method           dro   erm   jtt   rwg   rwy  subg  suby       dro        erm   
K name                                                                          
2 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.803151   0.987988   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  2.083051   3.961755   
  birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  1.269213   3.290191   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN   1.639344   
  medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  1.020621   1.020621   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  6.381711  13.354265   
  plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.914285   0.961235   
  resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  2.204921   4.391292   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  3.578916   3.868997   
4 AWA            NaN   NaN   5.0   NaN   NaN   NaN   5.0       NaN        NaN   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  9.682299   2.014056   
  birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.191201   0.332165   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN   1.906305   
  medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  1.289205   0.750537   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  6.056894   2.220001   
  plt-net        NaN   NaN   5.0   NaN   NaN   NaN   0.0       NaN        NaN   
  resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  5.312459   0.945211   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  5.076574   3.167154   
8 AWA            1.0   NaN   5.0   5.0   5.0   NaN   NaN       NaN        NaN   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.389759   0.948754   
  birds          NaN   5.0   5.0   5.0   5.0  10.0  10.0       NaN   0.205541   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN   2.586398   
  medical-leaf  15.0  15.0  15.0  15.0  15.0  15.0  15.0  0.593914   2.182179   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   NaN  1.859436   2.264674   
  plt-net        NaN  10.0   NaN   NaN   NaN   5.0   NaN       NaN   1.164963   
  resisc         3.0   NaN   5.0   5.0   5.0   5.0   5.0  4.006168        NaN   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.680414   1.039349   

                                                                   
method               jtt       rwg       rwy       subg      suby  
K name                                                             
2 AWA           0.491284  1.424136  0.920850   0.121414  2.655719  
  airplanes     1.008613  2.205686  3.282543   2.425670  1.068242  
  birds         0.941172  0.944602  4.070067   1.964652  4.939136  
  dogs          2.223715       NaN       NaN        NaN  8.640286  
  medical-leaf  3.486083  2.825971  1.020621   2.041241  2.429563  
  plt-doc       4.528302  3.077265  3.007052  11.779760  2.315733  
  plt-net       0.492173  0.467844  1.264302   0.981095  1.199565  
  resisc        1.916630  2.271783  1.931949   1.300619  1.840586  
  texture-dtd   1.619709  3.239418  8.932186   4.157397  2.222222  
4 AWA           0.352764       NaN       NaN        NaN  0.507287  
  airplanes     1.863330  2.154535  1.762424   1.068535  2.053270  
  birds         0.192649  0.453565  1.440390   0.439819  0.254035  
  dogs          1.482305       NaN       NaN        NaN  3.198555  
  medical-leaf  1.191113  1.047209  1.343549   5.818603  7.803703  
  plt-doc       5.958289  3.251940  3.660183   2.828631  2.802762  
  plt-net       1.406438       NaN       NaN        NaN       NaN  
  resisc        1.645443  1.112697  3.146463   4.796541  7.046332  
  texture-dtd   3.868997  3.685139  1.883981   1.416394  5.076574  
8 AWA           1.123868  1.552049  1.622404        NaN       NaN  
  airplanes     0.336402  1.772654  0.489716   1.351742  0.547073  
  birds         0.54

In [16]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## c. relative_acc

In [17]:
x_axis = 'K'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## d. minor_group_acc

In [18]:
x_axis = 'K'
y_axis = 'minor_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


# III. Analysis : mu

Here are the **plot parameters** to be played with :

In [19]:
ERROR_BARS = True #Wether to plot the error bars
NORMALIZE = True

To analyse the impact of mu, we the **value of K is fixed**:

In [20]:
df_fix_K = df[df['K']==4] # Currently K in [2,4,8,12]


## a. best_acc_te

In [21]:
x_axis = 'mu'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, normalize=NORMALIZE)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## b. worst_acc_te

In [22]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)

count                                           sem            \
method           dro   erm   jtt   rwg   rwy  subg  suby       dro       erm   
K name                                                                         
2 AWA            5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.096700  0.060566   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.120025  0.182957   
  birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.065045  0.165842   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN  0.226769   
  medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.055678  0.100000   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.120417  0.211749   
  plt-net        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.085572  0.158121   
  resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.112201  0.184009   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.063475  0.077521   
4 AWA            NaN   NaN   5.0   NaN   NaN   NaN   5.0       NaN       NaN   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.148833  0.064670   
  birds          5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.070757  0.223997   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN  0.213201   
  medical-leaf   5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.029705  0.022636   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.192769  0.144179   
  plt-net        NaN   NaN   0.0   NaN   NaN   NaN   0.0       NaN       NaN   
  resisc         5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.170567  0.085762   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.181407  0.109550   
8 AWA            1.0   NaN   5.0   5.0   5.0   NaN   NaN       NaN       NaN   
  airplanes      5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.179803  0.193048   
  birds          NaN   5.0   5.0   5.0   5.0  10.0  10.0       NaN  0.027524   
  dogs           NaN   5.0   5.0   NaN   NaN   NaN   5.0       NaN  0.224744   
  medical-leaf  15.0  15.0  15.0  15.0  15.0  15.0  15.0  0.026726  0.088904   
  plt-doc        5.0   5.0   5.0   5.0   5.0   5.0   NaN  0.157270  0.113044   
  plt-net        NaN  10.0   NaN   NaN   NaN   5.0   NaN       NaN  0.151457   
  resisc         3.0   NaN   5.0   5.0   5.0   5.0   5.0  0.109747       NaN   
  texture-dtd    5.0   5.0   5.0   5.0   5.0   5.0   5.0  0.065638  0.194365   

                                                                  
method               jtt       rwg       rwy      subg      suby  
K name                                                            
2 AWA           0.046218  0.159393  0.113985  0.024451  0.000000  
  airplanes     0.047619  0.091143  0.181433  0.102992  0.087312  
  birds         0.134097  0.074725  0.173341  0.079078  0.010327  
  dogs          0.184705       NaN       NaN       NaN  0.244949  
  medical-leaf  0.158114  0.188680  0.061237  0.156844  0.168523  
  plt-doc       0.085155  0.043422  0.049387  0.155986  0.100015  
  plt-net       0.061797  0.096745  0.097005  0.005729  0.105664  
  resisc        0.135543  0.137188  0.141238  0.070588  0.129897  
  texture-dtd   0.032814  0.058696  0.137389  0.000000  0.037719  
4 AWA           0.000000       NaN       NaN       NaN  0.000000  
  airplanes     0.177157  0.050103  0.087251  0.107997  0.175798  
  birds         0.079808  0.112495  0.222711  0.155582  0.178013  
  dogs          0.244949       NaN       NaN       NaN  0.201867  
  medical-leaf  0.032078  0.011563  0.067166  0.073237  0.105263  
  plt-doc       0.193781  0.183385  0.150824  0.159051  0.129961  
  plt-net            NaN       NaN       NaN       NaN       NaN  
  resisc        0.194071  0.074683  0.183139  0.175937  0.180169  
  texture-dtd   0.182696  0.170424  0.060921  0.033333  0.194416  
8 AWA           0.063158  0.105229  0.226078       NaN       NaN  
  airplanes     0.149606  0.198267  0.062966  0.107981  0.103383  
  birds         0.082731  0.089238  0.137078  0.062890  0.095240  
  dogs

In [23]:
x_axis = 'mu'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## c. relative_acc

In [24]:
x_axis = 'mu'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


# IV. Examples 

## a. Dataset difficulty :

In [25]:
def plot_violin(df, metric, category):
    """
    Creates and displays a violin plot for the given DataFrame.
    Parameters:
    - df: pandas DataFrame containing the data to plot.
    - metric: str, the name of the column in df containing the values to plot.
    - category: str, the name of the column in df representing different categories to separate the violins.

    """
    fig = go.Figure()
    categories = df[category].unique()
    for cat in categories:
        cat_data = df[df[category] == cat][metric]
        fig.add_trace(go.Violin(y=cat_data, name=cat, box_visible=True, meanline_visible=True))

    fig.update_layout(title=f"{metric} by {category} for dataset",
                      yaxis_title=metric,
                      legend_title=category)

    # Show the figure
    fig.show()

In [26]:
plot_violin(df_fix_mu,'best_acc_te','method')

In [27]:
plot_violin(df_fix_mu_K2,'worst_grp_acc_te','method')

In [28]:
plot_violin(df_fix_mu_K12,'worst_grp_acc_te','method')

In [29]:
plot_violin(df_fix_mu,'worst_grp_acc_te','method')

In [30]:
plot_violin(df_fix_mu,'best_acc_te','name')

In [31]:
plot_violin(df_fix_mu_K2,'best_acc_te','name')

In [32]:
plot_violin(df_fix_mu,'worst_grp_acc_te','name')

In [33]:
plot_violin(df_fix_mu_K12,'best_acc_te','name')

In [34]:
df_1 = df_fix_mu[df_fix_mu['name'] == 'plt-net']
df_1 = df_1[df_1['K'] == 2]
df_1 = df_1[df_1['method'] == 'erm']
df_1['best_acc_te']

1258    0.962852
1259    0.956296
1261    0.959437
1263    0.956706
1265    0.958891
Name: best_acc_te, dtype: float64

In [35]:
df_2 = df_fix_mu[df_fix_mu['name'] == 'resisc']
df_2 = df_2[df_2['K'] == 2]
df_2 = df_2[df_2['method'] == 'erm']
df_2['best_acc_te']

1593    0.990476
1594    0.972619
1595    0.963095
1596    0.982143
1597    0.970238
Name: best_acc_te, dtype: float64

In [36]:
df_2['best_acc_te'].std()/(5)**.5 # We check the computation from previous error bar in graphs

0.0047856550576766185